In [1]:
import numpy as np
import random
import operator 
import pandas as pd 
import matplotlib.pyplot as plt
import math

In [2]:
def routeFitness(route, dm):
    dist = 0
    for i in range(len(route)-1):
        dist += dm[route[i], route[i+1]]
    dist += dm[route[-1], route[0]]
    fitness = 1/dist
    
    return fitness

In [3]:
def dist_matrix(df):
    list_of_distances = [] # just to make it simple, probably there are 1k better ways to do this
    for point in df.itertuples():
        a = point[1]
        b = point[2]
        for point_s in df.itertuples():
            c = point_s[1]
            d = point_s[2]
            dist = math.sqrt(((c-a)**2) + ((d-b)**2))
            list_of_distances.append(dist)
    dist_matrix = np.array(list_of_distances) # convertion to array
    dist_matrix = np.reshape(dist_matrix, (len(df), len(df))) # reshaping
    # dist_matrix.shape # check
    return dist_matrix

In [4]:
def initPopulation(popSize, cities):
    population = []
    [population.append(random.sample(cities, len(cities))) for i in range(0, popSize)]
#     for i in range(0, popSize):
#         population.append(createRoute(cityList))
    return population

L'errore si crea alla fine della prima iterazione, succede qualcosa per cui non si sovrascrive correttamente la nuova popolazione

In [63]:
def GenA(permutation, dm, popSize, elite_n, mutation_r, n_of_gen):
    
    pop = initPopulation(popSize, permutation)
    fitness = {}
    [fitness.update({i:routeFitness(pop[i], dm)}) for i in range(0,len(pop))]
    rankedRoutes = sorted(fitness.items(), key = operator.itemgetter(1), reverse = True)
    print("Initial distance: " + str(1 / rankedRoutes[0][1]))
    del rankedRoutes
    del fitness
    
    for k in range(0, n_of_gen):
        
        ##################
        # Ranking routes #
        ##################
        
        print(k)
        fitness = None
        fitness = {}
        
        #print(k)
        
        [fitness.update({i:routeFitness(pop[i], dm)}) for i in range(0,len(pop))]

#       for i in range(0, len(pop)):
# #         # fitnessResults[i] = Fitness(population[i], dm).routeFitness()
#             fitness[i] = routeFitness(pop[i], dm)
#        # print(fitness)
        
        #print(rankedRoutes)
        rankedRoutes = None
        #print(fitness)
        rankedRoutes = sorted(fitness.items(), key = lambda kv: kv[1], reverse = True) ## il problema si crea qui
        #print(rankedRoutes)
    
#         rankedRoutes = rankRoutes(pop, dm)

        #############
        # selection #
        #############
        
        selectionR = []
        df = pd.DataFrame(np.array(rankedRoutes), columns=["Index","Fitness"])
        df['cum_sum'] = df.Fitness.cumsum()
        df['cum_perc'] = 100*df.cum_sum/df.Fitness.sum()
        [selectionR.append(rankedRoutes[i][0]) for i in range(0, elite_n)]
        
        for i in range(0, len(rankedRoutes) - elite_n):
            epsilon = 100*random.random()
            for i in range(0, len(rankedRoutes)):
                if epsilon <= df.iat[i,3]:
                    selectionR.append(rankedRoutes[i][0])
                    break  
    
        #print(selectionR)
        
        ###############
        # mating pool #
        ###############
        
        mp = []
       # [mp.append(pop[selectionR[i]]) for i in range(0, len(selectionR))]
        for i in range(0, len(selectionR)):
            index = selectionR[i]
            mp.append(pop[index])


            
        #############
        # Crossover #
        #############
        
        children = []
        length = (len(mp) - elite_n)
        pool = random.sample(mp, len(mp))
    
        [children.append(mp[i]) for i in range(0, elite_n)]
        
        for i in range(0, length):
            child = []
            childP1 = []
            childP2 = []
    
            geneA = int(random.random() * len(pool[i]))
            geneB = int(random.random() * len(pool[i]))
            
            startGene = min(geneA, geneB)
            endGene = max(geneA, geneB)
            [childP1.append(pool[i]) for i in range(startGene, endGene)]
            childP2 = [item for item in pool[len(mp)-i-1] if item not in childP1]
            
            child = childP1 + childP2
            
            children.append(child)
        
        #print(children)    
        
        ############
        # mutation #
        ############
        
        mutatedPop = []
    
        for ind in range(0, len(children)):
            individual = children[ind]
            for swapped in range(len(individual)):
                if(random.random() < mutation_r):
                    swapWith = int(random.random() * len(individual))
            
                    city1 = individual[swapped]
                    city2 = individual[swapWith]
            
                    individual[swapped] = city2
                    individual[swapWith] = city1
                    #print(individual)
            mutatedPop.append(individual)
        
        #print(len(pop), len(mutatedPop), len(children), len(mp), len(pool), length, len(selectionR), len(rankedRoutes))
       # print(mutatedPop)
        
        ##############
        # update pop #
        ##############
       # print(mutatedPop)
        print(mutatedPop)
        pop = mutatedPop
        
    
    fitness = None
    fitness = {}
    [fitness.update({i:routeFitness(pop[i], dm)}) for i in range(0,len(pop))]
    # print(fitness)
    rankedRoutes = None
    rankedRoutes = sorted(fitness.items(), key = lambda kv: kv[1], reverse = True)  
    print("Final distance: " + str(1 / rankedRoutes[0][1]))
    bestRouteIndex = rankedRoutes[0][0]
    bestRoute = pop[bestRouteIndex]
    return bestRoute

In [307]:
GenA(permutation = cities, popSize = 100, dm = d_matrix, elite_n = 5, mutation_r = 0.01, n_of_gen = 100)

Initial distance: 26039.418140805086
0
[[4, 46, 28, 15, 51, 12, 24, 16, 10, 23, 1, 17, 21, 32, 26, 13, 2, 47, 39, 34, 22, 14, 29, 44, 9, 18, 40, 8, 3, 19, 45, 20, 49, 41, 43, 6, 50, 27, 5, 37, 33, 7, 25, 38, 11, 48, 36, 31, 0, 35, 42, 30], [4, 31, 39, 3, 41, 43, 18, 25, 38, 32, 17, 2, 29, 33, 16, 9, 6, 0, 50, 10, 26, 40, 19, 44, 49, 15, 34, 30, 37, 51, 46, 13, 47, 48, 28, 5, 14, 8, 7, 21, 45, 1, 27, 24, 42, 35, 20, 22, 36, 23, 11, 12], [6, 13, 46, 50, 49, 21, 7, 38, 44, 26, 51, 0, 43, 36, 2, 48, 33, 17, 25, 16, 1, 19, 18, 30, 4, 39, 47, 37, 31, 8, 20, 45, 12, 23, 15, 28, 11, 27, 29, 3, 22, 14, 24, 40, 34, 5, 9, 35, 32, 42, 10, 41], [15, 10, 13, 26, 19, 34, 1, 22, 33, 3, 44, 18, 31, 7, 8, 29, 28, 46, 36, 21, 50, 9, 41, 16, 32, 4, 30, 37, 5, 27, 45, 47, 40, 23, 24, 38, 42, 2, 49, 48, 0, 14, 51, 12, 39, 43, 20, 25, 17, 35, 11, 6], [18, 20, 0, 50, 48, 14, 5, 10, 27, 47, 35, 30, 44, 2, 4, 12, 25, 6, 26, 17, 21, 38, 1, 46, 34, 31, 40, 32, 33, 39, 36, 22, 15, 49, 24, 29, 37, 8, 51, 13, 42, 3,

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [7]:
df = pd.read_csv("berlin52.tsp", header=None, sep = " ", names = ['x', 'y'], index_col = 0)
df.head()
cities = []
cities = list(range(len(df)))
d_matrix = dist_matrix(df)

In [627]:
perm = [15,
 30,
 5,
 50,
 22,
 33,
 31,
 2,
 19,
 46,
 41,
 36,
 23,
 27,
 32,
 25,
 12,
 1,
 4,
 42,
 40,
 9,
 47,
 10,
 44,
 13,
 43,
 6,
 20,
 34,
 45,
 28,
 21,
 16,
 37,
 48,
 38,
 7,
 0,
 39,
 3,
 18,
 14,
 11,
 26,
 51,
 8,
 49,
 17,
 35,
 29,
 24]

In [629]:
1/routeFitness(perm, d_matrix)

27582.012577833248